<a href="https://colab.research.google.com/github/mohak1/DDoS_Attack_Classification/blob/master/DDOS_SGD%2C_KNN%2C_Naive_Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
cd ./drive/My\ Drive/dos_dataset/

/content/drive/My Drive/dos_dataset


In [0]:
import pandas as pd
df = pd.read_csv("./clean_5.csv")
#df.info()
df=df.drop('Unnamed: 0.1',axis=1)
x=df.iloc[:,df.columns != 'Label']
y=df.iloc[:,-1]
print("x\n",x.info())
y = pd.DataFrame(y)
print('y\n',y.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1245798 entries, 0 to 1245797
Data columns (total 38 columns):
Unnamed: 0                     1245798 non-null int64
Unnamed: 0.1.1                 1245798 non-null int64
 Source Port                   1245798 non-null int64
 Destination IP                1245798 non-null int64
 Destination Port              1245798 non-null int64
 Protocol                      1245798 non-null int64
Total Length of Fwd Packets    1245798 non-null float64
 Fwd Packet Length Max         1245798 non-null float64
 Fwd Packet Length Min         1245798 non-null float64
 Fwd Packet Length Mean        1245798 non-null float64
Bwd Packet Length Max          1245798 non-null float64
 Bwd Packet Length Min         1245798 non-null float64
 Bwd Packet Length Mean        1245798 non-null float64
 Bwd Packet Length Std         1245798 non-null float64
Flow Bytes/s                   1245753 non-null float64
 Flow Packets/s                1245798 non-null float64
 Fl

In [0]:
df.describe()

,Unnamed: 0,Unnamed: 0.1.1,Source Port,Destination IP,Destination Port,Protocol,Total Length of Fwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Fwd IAT Mean,Fwd IAT Std,Fwd PSH Flags,Fwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Variance,RST Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Subflow Fwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,Idle Std,Inbound,Label
count,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245753e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06
mean,6.228985e+05,1.074318e+05,2.302203e+04,3.166084e+09,3.186662e+04,1.486217e+01,1.069863e+03,4.679689e+02,4.562818e+02,4.597351e+02,1.475077e+01,1.725317e+00,5.181360e+00,3.952797e+00,inf,inf,2.181317e+05,3.961819e+05,2.936620e+05,4.491561e+05,8.118491e-03,1.228033e+06,4.562034e+02,4.765040e+02,4.608991e+02,2.453121e+03,8.118491e-03,1.699826e-01,2.298206e-02,1.019186e-02,6.812989e+02,4.597351e+02,5.181360e+00,1.069863e+03,1.205828e+03,1.214158e+02,1.914283e+05,9.624289e-01,3.339989e+00
std,3.596310e+05,1.089688e+05,2.447340e+04,4.198623e+08,1.958453e+04,4.363049e+00,1.441733e+03,4.837410e+02,4.815043e+02,4.801285e+02,1.730104e+02,1.246812e+01,5.426732e+01,5.142089e+01,NaN,NaN,1.008884e+06,1.777128e+06,1.293623e+06,2.018263e+06,8.973621e-02,9.192624e+05,4.814801e+02,5.001346e+02,4.795755e+02,3.517655e+04,8.973621e-02,3.756177e-01,1.498463e-01,1.004390e-01,7.221197e+02,4.801285e+02,5.426732e+01,1.441733e+03,3.331159e+03,2.118996e+03,1.175897e+06,1.901566e-01,1.832971e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,6.724045e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.571637e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,-1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,3.114492e+05,2.503600e+04,7.690000e+02,3.232248e+09,1.479500e+04,1.700000e+01,4.580000e+02,2.290000e+02,2.290000e+02,2.290000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,9.541667e+06,4.166667e+04,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,3.921569e+04,2.290000e+02,2.290000e+02,2.290000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.435000e+02,2.290000e+02,0.000000e+00,4.580000e+02,-1.000000e+00,-1.000000e+00,0.000000e+00,1.000000e+00,2.000000e+00
50%,6.228985e+05,7.408800e+04,9.447000e+03,3.232248e+09,3.190300e+04,1.700000e+01,4.940000e+02,2.470000e+02,2.470000e+02,2.470000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.580000e+08,2.000000e+06,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,2.000000e+06,2.470000e+02,2.470000e+02,2.470000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.705000e+02,2.470000e+02,0.000000e+00,4.940000e+02,-1.000000e+00,-1.000000e+00,0.000000e+00,1.000000e+00,3.000000e+00
75%,9.343478e+05,1.462510e+05,4.755075e+04,3.232248e+09,4.894600e+04,1.700000e+01,1.398000e+03,4.840000e+02,4.720000e+02,4.720000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,9.813333e+08,2.000000e+06,4.500000e+01,0.000000e+00,3.000000e+00,0.000000e+00,0.000000e+00,2.000000e+06,4.720000e+0

In [0]:
#normalizating the data
#df.iloc[:,df.columns != ' Flow Packets/s' or df.columns != 'Flow Bytes/s']
normalized_df=(df-df.mean())/df.std()
normalized_df=normalized_df.drop(' Flow Packets/s',axis=1)
normalized_df=normalized_df.drop('Flow Bytes/s',axis=1)

In [0]:
import matplotlib.pyplot as plt
import seaborn as sn
plt.figure(figsize=(100,200))
cor = normalized_df.corr()
sn.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.show()

In [0]:
#df=df.drop('Unnamed: 0.1',axis=1)
normalized_df.head()

,Unnamed: 0,Unnamed: 0.1.1,Source Port,Destination IP,Destination Port,Protocol,Total Length of Fwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow IAT Mean,Flow IAT Std,Fwd IAT Mean,Fwd IAT Std,Fwd PSH Flags,Fwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Variance,RST Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Subflow Fwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,Idle Std,Inbound,Label
0,-1.732049,0.186725,1.395596,0.157586,-0.607399,-2.031188,-0.725421,-0.954992,-0.935156,-0.945029,-0.050579,0.342849,0.015085,-0.076871,-0.178761,-0.175445,-0.178330,-0.168507,-0.090471,-1.335866,-0.935040,-0.940755,-0.948545,-0.069737,-0.090471,2.209740,-0.153371,-0.101473,-0.933777,-0.945029,0.015085,-0.725421,-0.054584,-0.057299,-0.162793,0.19758,-0.731047
1,-1.732046,-0.373912,0.347887,0.157586,-1.625958,-2.031188,-0.733744,-0.954992,-0.935156,-0.945029,-0.050579,0.342849,0.015085,-0.076871,-0.216151,-0.222876,-0.227006,-0.222546,-0.090471,-1.324001,-0.935040,-0.940755,-0.948545,-0.069737,-0.090471,2.209740,-0.153371,-0.101473,-0.933085,-0.945029,0.015085,-0.733744,7.027636,-0.057299,-0.162793,0.19758,-0.731047
2,-1.732043,0.213485,1.196809,0.157586,-0.560525,-2.031188,-0.725421,-0.954992,-0.935156,-0.945029,-0.050579,0.342849,0.015085,-0.076871,-0.182709,-0.180450,-0.183461,-0.174204,-0.090471,-1.335864,-0.935040,-0.940755,-0.948545,-0.069737,-0.090471,2.209740,-0.153371,-0.101473,-0.933777,-0.945029,0.015085,-0.725421,-0.054584,-0.057299,-0.162793,0.19758,-0.731047
3,-1.732040,0.215559,0.892029,0.157586,-1.562081,-2.031188,-0.725421,-0.954992,-0.935156,-0.945029,-0.050579,0.342849,0.015085,-0.076871,-0.186161,-0.184840,-0.187948,-0.179186,-0.090471,-1.335861,-0.935040,-0.940755,-0.948545,-0.069737,-0.090471,2.209740,-0.153371,-0.101473,-0.933777,-0.945029,0.015085,-0.725421,-0.054584,-0.057299,-0.162793,0.19758,-0.731047
4,-1.732038,-0.757481,-0.914218,0.157586,0.169285,0.489985,-0.424394,-0.494002,-0.472024,-0.480570,-0.085259,-0.138378,-0.095478,-0.076871,-0.216208,-0.222934,-0.227005,-0.222546,-0.090471,-0.610670,-0.471885,-0.494875,-0.483551,-0.069737,-0.090471,-0.452542,-0.153371,-0.101473,-0.467788,-0.480570,-0.095478,-0.424394,-0.362285,-0.057771,-0.162793,0.19758,-0.731047


In [0]:
normalized_df=normalized_df.drop('Unnamed: 0', axis=1)
normalized_df=normalized_df.drop('Unnamed: 0.1.1', axis=1) 

In [0]:
y=pd.DataFrame(y)

In [0]:
normalized_df.describe()

,Source Port,Destination IP,Destination Port,Protocol,Total Length of Fwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow IAT Mean,Flow IAT Std,Fwd IAT Mean,Fwd IAT Std,Fwd PSH Flags,Fwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Variance,RST Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Subflow Fwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,Idle Std,Inbound,Label
count,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06
mean,1.054733e-12,1.373855e-13,-4.158871e-15,-1.240177e-11,1.063288e-13,-8.237663e-13,2.961019e-12,-1.186983e-12,8.682086e-13,9.755238e-13,-8.253327e-13,-1.077336e-12,2.138927e-12,2.701441e-12,-2.322818e-12,1.352296e-12,7.837405e-13,1.950589e-12,3.237758e-12,1.526851e-12,-1.568473e-12,-7.298400e-13,7.837405e-13,-3.928092e-12,-2.100294e-12,-3.022469e-13,1.306471e-12,-1.186983e-12,-8.253385e-13,1.063288e-13,1.868235e-12,1.265760e-13,-1.166242e-12,-2.792804e-12,-4.782956e-12
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-9.406960e-01,-7.380619e+00,-1.627133e+00,-3.406373e+00,-7.420674e-01,-9.673956e-01,-9.476172e-01,-9.575252e-01,-8.525948e-02,-1.383783e-01,-9.547846e-02,-7.687141e-02,-2.162110e-01,-2.229338e-01,-2.270074e-01,-2.225459e-01,-9.047062e-02,-1.335889e+00,-9.475020e-01,-9.527515e-01,-9.610563e-01,-6.973740e-02,-9.047062e-02,-4.525416e-01,-1.533709e-01,-1.014731e-01,-9.434708e-01,-9.575252e-01,-9.547846e-02,-7.420674e-01,-3.622846e-01,-5.777066e-02,-1.627934e-01,-5.061243e+00,-1.822172e+00
25%,-9.092742e-01,1.575856e-01,-8.716894e-01,4.899846e-01,-4.243941e-01,-4.940017e-01,-4.720244e-01,-4.805696e-01,-8.525948e-02,-1.383783e-01,-9.547846e-02,-7.687141e-02,-2.162100e-01,-2.229338e-01,-2.270067e-01,-2.225459e-01,-9.047062e-02,-1.293230e+00,-4.718853e-01,-4.948748e-01,-4.835507e-01,-6.973740e-02,-9.047062e-02,-4.525416e-01,-1.533709e-01,-1.014731e-01,-4.677879e-01,-4.805696e-01,-9.547846e-02,-4.243941e-01,-3.622846e-01,-5.777066e-02,-1.627934e-01,1.975798e-01,-7.310472e-01
50%,-5.546851e-01,1.575856e-01,1.857374e-03,4.899846e-01,-3.994241e-01,-4.567917e-01,-4.346415e-01,-4.430796e-01,-8.525948e-02,-1.383783e-01,-9.547846e-02,-7.687141e-02,-2.162100e-01,-2.229338e-01,-2.270067e-01,-2.225459e-01,-9.047062e-02,8.397680e-01,-4.345006e-01,-4.588845e-01,-4.460175e-01,-6.973740e-02,-9.047062e-02,-4.525416e-01,-1.533709e-01,-1.014731e-01,-4.303980e-01,-4.430796e-01,-9.547846e-02,-3.994241e-01,-3.622846e-01,-5.777066e-02,-1.627934e-01,1.975798e-01,-1.854849e-01
75%,1.002261e+00,1.575856e-01,8.720852e-01,4.899846e-01,2.275992e-01,3.313989e-02,3.264403e-02,2.554496e-02,-8.525948e-02,-1.383783e-01,-9.547846e-02,-7.687141e-02,-2.161664e-01,-2.229338e-01,-2.270051e-01,-2.225459e-01,-9.047062e-02,8.397680e-01,3.280845e-02,1.698744e-02,2.731780e-02,-6.973740e-02,-9.047062e-02,-4.525416e-01,-1.533709e-01,-1.014731e-01,3.697599e-02,2.554496e-02,-9.547846e-02,2.275992e-01,-3.622846e-01,-5.777066e-02,-1.627934e-01,1.975798e-01,9.056397e-01
max,1.737069e+00,2.688699e+00,1.719132e+00,4.8

In [0]:
normalized_x=normalized_df.iloc[:,normalized_df.columns != 'Label']

In [0]:
normalized_x.describe()

,Source Port,Destination IP,Destination Port,Protocol,Total Length of Fwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow IAT Mean,Flow IAT Std,Fwd IAT Mean,Fwd IAT Std,Fwd PSH Flags,Fwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Variance,RST Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Subflow Fwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,Idle Std,Inbound
count,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06
mean,1.054733e-12,1.373855e-13,-4.158871e-15,-1.240177e-11,1.063288e-13,-8.237663e-13,2.961019e-12,-1.186983e-12,8.682086e-13,9.755238e-13,-8.253327e-13,-1.077336e-12,2.138927e-12,2.701441e-12,-2.322818e-12,1.352296e-12,7.837405e-13,1.950589e-12,3.237758e-12,1.526851e-12,-1.568473e-12,-7.298400e-13,7.837405e-13,-3.928092e-12,-2.100294e-12,-3.022469e-13,1.306471e-12,-1.186983e-12,-8.253385e-13,1.063288e-13,1.868235e-12,1.265760e-13,-1.166242e-12,-2.792804e-12
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-9.406960e-01,-7.380619e+00,-1.627133e+00,-3.406373e+00,-7.420674e-01,-9.673956e-01,-9.476172e-01,-9.575252e-01,-8.525948e-02,-1.383783e-01,-9.547846e-02,-7.687141e-02,-2.162110e-01,-2.229338e-01,-2.270074e-01,-2.225459e-01,-9.047062e-02,-1.335889e+00,-9.475020e-01,-9.527515e-01,-9.610563e-01,-6.973740e-02,-9.047062e-02,-4.525416e-01,-1.533709e-01,-1.014731e-01,-9.434708e-01,-9.575252e-01,-9.547846e-02,-7.420674e-01,-3.622846e-01,-5.777066e-02,-1.627934e-01,-5.061243e+00
25%,-9.092742e-01,1.575856e-01,-8.716894e-01,4.899846e-01,-4.243941e-01,-4.940017e-01,-4.720244e-01,-4.805696e-01,-8.525948e-02,-1.383783e-01,-9.547846e-02,-7.687141e-02,-2.162100e-01,-2.229338e-01,-2.270067e-01,-2.225459e-01,-9.047062e-02,-1.293230e+00,-4.718853e-01,-4.948748e-01,-4.835507e-01,-6.973740e-02,-9.047062e-02,-4.525416e-01,-1.533709e-01,-1.014731e-01,-4.677879e-01,-4.805696e-01,-9.547846e-02,-4.243941e-01,-3.622846e-01,-5.777066e-02,-1.627934e-01,1.975798e-01
50%,-5.546851e-01,1.575856e-01,1.857374e-03,4.899846e-01,-3.994241e-01,-4.567917e-01,-4.346415e-01,-4.430796e-01,-8.525948e-02,-1.383783e-01,-9.547846e-02,-7.687141e-02,-2.162100e-01,-2.229338e-01,-2.270067e-01,-2.225459e-01,-9.047062e-02,8.397680e-01,-4.345006e-01,-4.588845e-01,-4.460175e-01,-6.973740e-02,-9.047062e-02,-4.525416e-01,-1.533709e-01,-1.014731e-01,-4.303980e-01,-4.430796e-01,-9.547846e-02,-3.994241e-01,-3.622846e-01,-5.777066e-02,-1.627934e-01,1.975798e-01
75%,1.002261e+00,1.575856e-01,8.720852e-01,4.899846e-01,2.275992e-01,3.313989e-02,3.264403e-02,2.554496e-02,-8.525948e-02,-1.383783e-01,-9.547846e-02,-7.687141e-02,-2.161664e-01,-2.229338e-01,-2.270051e-01,-2.225459e-01,-9.047062e-02,8.397680e-01,3.280845e-02,1.698744e-02,2.731780e-02,-6.973740e-02,-9.047062e-02,-4.525416e-01,-1.533709e-01,-1.014731e-01,3.697599e-02,2.554496e-02,-9.547846e-02,2.275992e-01,-3.622846e-01,-5.777066e-02,-1.627934e-01,1.975798e-01
max,1.737069e+00,2.688699e+00,1.719132e+00,4.899846e-01,1.438922e+02,6.526284e+00,3.478096e+00,3.480870e+00,2.087880e+01,1.169603e+02,3.400920e+01,

In [0]:
from sklearn.model_selection import train_test_split
import numpy as np
X_train, X_test, y_train, y_test = train_test_split(normalized_x, y, test_size=0.25, random_state=0)
y_train = np.array(y_train)

In [0]:
normalized_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1245798 entries, 0 to 1245797
Data columns (total 34 columns):
 Source Port                   1245798 non-null float64
 Destination IP                1245798 non-null float64
 Destination Port              1245798 non-null float64
 Protocol                      1245798 non-null float64
Total Length of Fwd Packets    1245798 non-null float64
 Fwd Packet Length Max         1245798 non-null float64
 Fwd Packet Length Min         1245798 non-null float64
 Fwd Packet Length Mean        1245798 non-null float64
Bwd Packet Length Max          1245798 non-null float64
 Bwd Packet Length Min         1245798 non-null float64
 Bwd Packet Length Mean        1245798 non-null float64
 Bwd Packet Length Std         1245798 non-null float64
 Flow IAT Mean                 1245798 non-null float64
 Flow IAT Std                  1245798 non-null float64
 Fwd IAT Mean                  1245798 non-null float64
 Fwd IAT Std                   1245798 non-null

In [0]:
y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1245798 entries, 0 to 1245797
Data columns (total 1 columns):
Label    1245798 non-null int64
dtypes: int64(1)
memory usage: 9.5 MB


Stochastic Gradient Descent

In [0]:
#stochastic gradient descent
from  sklearn.linear_model import SGDClassifier, LinearRegression
from sklearn import metrics
lr = SGDClassifier(loss="log", penalty="l2", max_iter=200)
lr.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=200,
              n_iter_no_change=5, n_jobs=None, penalty='l2', power_t=0.5,
              random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [0]:
y_pred = lr.predict(X_test)
print("Stochastic Gradient Descent\nAccuracy =", metrics.accuracy_score(y_test, y_pred)*100)

Stochastic Gradient Descent
Accuracy = 76.68903515813132


In [0]:
from sklearn import metrics
from sklearn.metrics import f1_score
print('Stochastic Gradient Descent')

print('Accuracy = ', metrics.accuracy_score(y_test, y_pred)*100)
print("Confusion Matrix =\n", metrics.confusion_matrix(y_test, y_pred, labels=None, 
                                              sample_weight=None))
print("Recall =", metrics.recall_score(y_test, y_pred, labels=None, 
                                             pos_label=1, average='weighted', 
                                             sample_weight=None))
print("Classification Report =\n", metrics.classification_report(y_test, y_pred, 
                                                                 labels=None, 
                                                                 target_names=None, 
                                                                 sample_weight=None, 
                                                                 digits=2, 
                                                                 output_dict=False))

print("F1 Score = ",f1_score(y_test, y_pred, average='macro'))

Stochastic Gradient Descent
Accuracy =  76.68903515813132
Confusion Matrix =
 [[14105    51     0     0    25    65    38    37]
 [    0 36664     2     1 13082     2    11     0]
 [    3     9 49422   363    17     7     1     0]
 [    0     1   292 47515   839     1  1626     1]
 [   24 34854     0   124 11761   116    33     6]
 [    9     0     0     4     0 49651     0     0]
 [    2     0     0 20457    13     4 29682     9]
 [   23     0     0   229    41    20   160    48]]
Recall = 0.7668903515813132
Classification Report =
               precision    recall  f1-score   support

           0       1.00      0.98      0.99     14321
           1       0.51      0.74      0.60     49762
           2       0.99      0.99      0.99     49822
           3       0.69      0.95      0.80     50275
           4       0.46      0.25      0.32     46918
           5       1.00      1.00      1.00     49664
           6       0.94      0.59      0.73     50167
           7       0.48    

In [0]:
yt = y_test.iloc[:,0]
yt=np.array(yt)

In [0]:
y_pred[3]

2

In [0]:
y_test = yt
#y_pred = y_pred[:, 0]

KNN Classifier

In [0]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=8)
model.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=8, p=2,
                     weights='uniform')

In [0]:
y_pred = model.predict(X_test)
print("Accuracy =", metrics.accuracy_score(y_test, y_pred))

Accuracy = 0.8700979290415797


In [0]:
from sklearn import metrics
from sklearn.metrics import f1_score

print('K Nearest Neighbour Classifier')

print('Accuracy = ', metrics.accuracy_score(y_test, y_pred)*100)
print("Confusion Matrix =\n", metrics.confusion_matrix(y_test, y_pred, labels=None, 
                                              sample_weight=None))
print("Recall =", metrics.recall_score(y_test, y_pred, labels=None, 
                                             pos_label=1, average='weighted', 
                                             sample_weight=None))
print("Classification Report =\n", metrics.classification_report(y_test, y_pred, 
                                                                 labels=None, 
                                                                 target_names=None, 
                                                                 sample_weight=None, 
                                                                  digits=2, 
                                                                 output_dict=False))
print("F1 Score = ",f1_score(y_test, y_pred, average='macro'))

K Nearest Neighbour Classifier
Accuracy =  87.00979290415796
Confusion Matrix =
 [[14317     0     0     0     2     0     2     0]
 [    1 36944     2    12 12802     1     0     0]
 [    3     4 49750    56     6     3     0     0]
 [    0     0    54 48840    26     0  1354     1]
 [    8 24436    11    67 22379     1     6    10]
 [    6     0     0     1     1 49654     2     0]
 [    1     0     0  1200    12     1 48915    38]
 [    2     0     0    15    39     2   270   193]]
Recall = 0.8700979290415797
Classification Report =
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     14321
           1       0.60      0.74      0.66     49762
           2       1.00      1.00      1.00     49822
           3       0.97      0.97      0.97     50275
           4       0.63      0.48      0.54     46918
           5       1.00      1.00      1.00     49664
           6       0.97      0.98      0.97     50167
           7       0.80 

Naive Bayes

In [0]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:206: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GaussianNB(priors=None, var_smoothing=1e-09)

In [0]:
pred = model.predict(X_test)

In [0]:
from sklearn import metrics
from sklearn.metrics import f1_score

print('Naive Bayes')

print('Accuracy = ', metrics.accuracy_score(y_test, pred)*100)
print("Confusion Matrix =\n", metrics.confusion_matrix(y_test, y_pred, labels=None, 
                                              sample_weight=None))
print("Recall =", metrics.recall_score(y_test, y_pred, labels=None, 
                                             pos_label=1, average='weighted', 
                                             sample_weight=None))
print("Classification Report =\n", metrics.classification_report(y_test, y_pred, 
                                                                 labels=None, 
                                                                 target_names=None, 
                                                                 sample_weight=None, 
                                                                  digits=2, 
                                                                 output_dict=False))
print("F1 Score = ",f1_score(y_test, y_pred, average='macro'))

Naive Bayes
Accuracy =  78.31658372130357
Confusion Matrix =
 [[14317     0     0     0     2     0     2     0]
 [    1 31901     2    10 17847     1     0     0]
 [    3     3 49751    55     7     3     0     0]
 [    0     0    52 48674    30     0  1517     2]
 [    8 20635    10    55 26188     1     7    14]
 [    6     0     0     1     1 49654     2     0]
 [    1     0     0  1076    14     1 49022    53]
 [    2     0     0    15    36     2   252   214]]
Recall = 0.8660170171777172
Classification Report =
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     14321
           1       0.61      0.64      0.62     49762
           2       1.00      1.00      1.00     49822
           3       0.98      0.97      0.97     50275
           4       0.59      0.56      0.58     46918
           5       1.00      1.00      1.00     49664
           6       0.96      0.98      0.97     50167
           7       0.76      0.41      0.53